In [52]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from xgboost import XGBClassifier

In [48]:

def read_data(file_path):
    data = pd.read_csv(file_path, encoding='latin-1')
    data.dropna(inplace=True)

    label_encoder = LabelEncoder()
    T_vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1, 2), max_features=5000)

    X = data['Email Text'].values
    y = data['Email Type'].values

    for i in range(len(y)):
        if y[i] == 'Phishing Email':
            y[i] = 1
        else:
            y[i] = 0

    X = T_vectorizer.fit_transform(X)
    y = label_encoder.fit_transform(y)
    return X, y


def print_report(y_val, y_pred, fold):
    print(f'Fold: {fold}')
    print(f'Accuracy Score: {accuracy_score(y_val, y_pred)}')
    print(f'Confusion Matrix: \n {confusion_matrix(y_val, y_pred)}')
    print(f'Classification Report: \n {classification_report(y_val, y_pred)}')


In [22]:
# data set preview
data = pd.read_csv('datasets/phishing_emails/phishing_email.csv')
print(data.head())
print(data['Email Type'].value_counts())

# X, y = read_data('datasets/phishing_emails/phishing_email.csv')
# print(X[0].shape, y.shape)

   Unnamed: 0                                         Email Text  \
0           0  re : 6 . 1100 , disc : uniformitarianism , re ...   
1           1  the other side of * galicismos * * galicismo *...   
2           2  re : equistar deal tickets are you still avail...   
3           3  \nHello I am your hot lil horny toy.\n    I am...   
4           4  software at incredibly low prices ( 86 % lower...   

       Email Type  
0      Safe Email  
1      Safe Email  
2      Safe Email  
3  Phishing Email  
4  Phishing Email  
Safe Email        11322
Phishing Email     7328
Name: Email Type, dtype: int64
(1, 10000) (18634,)


In [51]:
X, y = read_data('datasets/phishing_emails/phishing_email.csv')
num_folds = 5
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

print(X.shape, y.shape)

(18634, 5000) (18634,)


In [50]:
# k-fold experiment using XGBoost
fold = 1

for train_index, val_index in kfold.split(X):

  X_train, X_val = X[train_index], X[val_index]
  y_train, y_val = y[train_index], y[val_index]

  model = XGBClassifier(n_estimators=200, learning_rate=0.5, max_depth=4, colsample_bytree=0.5, n_jobs=-1, random_state=42)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_val)

  print_report(y_val, y_pred, fold)
  fold += 1



Fold: 1
Accuracy Score: 0.9659243359270191
Confusion Matrix: 
 [[2109  100]
 [  27 1491]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.99      0.95      0.97      2209
           1       0.94      0.98      0.96      1518

    accuracy                           0.97      3727
   macro avg       0.96      0.97      0.96      3727
weighted avg       0.97      0.97      0.97      3727

Fold: 2
Accuracy Score: 0.9691440837134424
Confusion Matrix: 
 [[2198   81]
 [  34 1414]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.98      0.96      0.97      2279
           1       0.95      0.98      0.96      1448

    accuracy                           0.97      3727
   macro avg       0.97      0.97      0.97      3727
weighted avg       0.97      0.97      0.97      3727

Fold: 3
Accuracy Score: 0.9643144620338073
Confusion Matrix: 
 [[2171   89]
 [  44 1423]]
Classification Report: 
  

In [47]:
# k-fold experiment using Adaboost with Decision Tree as base estimator
fold = 1

for train_index, val_index in kfold.split(X):

  X_train, X_val = X[train_index], X[val_index]
  y_train, y_val = y[train_index], y[val_index]

  tree = DecisionTreeClassifier(max_depth=4, max_features=500, random_state=42)
  model = AdaBoostClassifier(base_estimator=tree, n_estimators=400, random_state=42, learning_rate=0.1)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_val)

  print_report(y_val, y_pred, fold)
  fold += 1



Fold: 1
Accuracy Score: 0.951703783203649
Confusion Matrix: 
 [[2089  120]
 [  60 1458]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.97      0.95      0.96      2209
           1       0.92      0.96      0.94      1518

    accuracy                           0.95      3727
   macro avg       0.95      0.95      0.95      3727
weighted avg       0.95      0.95      0.95      3727



KeyboardInterrupt: 

In [ ]:
# k-fold experiment using random forest
fold = 1

for train_index, val_index in kfold.split(X):

  X_train, X_val = X[train_index], X[val_index]
  y_train, y_val = y[train_index], y[val_index]

  model = RandomForestClassifier(n_estimators=400, max_features=1000, random_state=42, n_jobs=-1)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_val)

  print_report(y_val, y_pred, fold)
  fold += 1  



In [ ]:
# Gradient boosting tree
fold = 1

for train_index, val_index in kfold.split(X):

  X_train, X_val = X[train_index], X[val_index]
  y_train, y_val = y[train_index], y[val_index]

  model = GradientBoostingClassifier(n_estimators=200, learning_rate=0.4, max_depth=3, random_state=42)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_val)

  print_report(y_val, y_pred, fold)
  fold += 1  